In [1]:
from random import random
from math import log
from math import sqrt
from math import exp

def bernoulli(VA):
    array = [(VA[ai], ai) for ai in VA]
    array.sort(reverse=True)
    
    sum=0
    new_array=[]
    for y,x in array:
        sum += y
        new_array.append((sum,x))

    def f():
        U = random()
        for y,x in new_array:
            if U > y:
                continue
            return x
    return f

def exp_k(k):
    def f():
        U = random()
        return -log(U)/k
    return f

def normal_standard():
    Exp = exp_k(1)
    sign = bernoulli({1:0.5, -1:0.5})
    def f():
        while True:
            Y = Exp()
            U = random()
            if U <= exp(-(Y - 1)**2/2) : return sign() * Y
    return f

def normal(u, o2):
    Z = normal_standard()
    return lambda: Z() * sqrt(o2) + u

In [2]:
get_arrival_airplane_time = exp_k(1/20)
get_time_of_loading_or_unloading = exp_k(1/30)
get_time_arrival = normal(10, 5)
get_time_takeoff = normal(10, 5)
get_repair_time = exp_k(1/15)
get_reload_time = exp_k(1/30)
has_rupture = bernoulli({0: 0.9, 1: 0.1})
do_loading_or_unloading = bernoulli({0: 0.5, 1: 0.5})

def get_time_of_airplane_in_line(): 
    t  = get_time_arrival()
    t += max( 
         get_reload_time(), 
         get_time_of_loading_or_unloading() if 1 == do_loading_or_unloading() else 0 
    )
    t += get_repair_time() if 1 == has_rupture() else 0
    t += get_time_takeoff()

    return t

l = []
for _ in range(10000):
    l.append(get_time_of_airplane_in_line())

print(sum(l)/ len(l))

58.83582886982672


In [3]:
class AeroportState: 
    def __init__(self) -> None:
        self.takeoff_time_in_path = [0] * 5
        self.time_free_of_path = [0] * 5 
        self.next_arrival_time = 0
        self.airplane_queue = []
        self.time = 0
    
    def get_free_path(self):
        if any(self.airplane_queue): raise IndexError
        return [i for i,time in enumerate(self.takeoff_time_in_path) if time <= self.time][0]
    
    def __str__(self) -> str:
        return f'{self.time_free_of_path} : {self.time}'

        

In [4]:
def arrival_event(state: AeroportState):
    if not state.time == state.next_arrival_time: return 
    state.next_arrival_time = state.time + get_time_arrival()
    # print(f'dispatch arrival event time({state.time}) new arrival generate time({state.next_arrival_time})')
    
    try:
        path_index = state.get_free_path()
        # print(f'the path {path_index} be free. Last takeoff in this path time({state.takeoff_time_in_path[path_index]})')
        state.time_free_of_path[path_index] += state.time - state.takeoff_time_in_path[path_index]
        # print(f'sum {state.time - state.takeoff_time_in_path[path_index]} to free time of path. Actually time({state.time_free_of_path[path_index]})')
        state.takeoff_time_in_path[path_index] = state.time + get_time_of_airplane_in_line()  
        # print(f'Next takeoff in path time({state.takeoff_time_in_path[path_index]})')
    except IndexError:
        state.airplane_queue.append(state.time)
        # print(f"Aeroport don't have free path, arrival add to queue. Queue.length = {len(state.airplane_queue)}")
        state.airplane_queue.sort()

    


In [5]:
def takeoff_event(state: AeroportState):
    if not state.time in state.takeoff_time_in_path: return 
    # print(f'dispatch takeoff event time({state.time}). Queue.length = {len(state.airplane_queue)}')

    if any(state.airplane_queue):
        index = state.takeoff_time_in_path.index(state.time)
        arrival_time = state.airplane_queue.pop(0)
        # print(f'the free path is the {index}. In this set the travel that arrival in time({arrival_time})')
        state.takeoff_time_in_path[index] =  state.time + get_time_of_airplane_in_line()
        # print(f'Next takeoff in the path be in time({state.takeoff_time_in_path[index]})')
    


In [8]:
import time as Time

def aeroport_simulate(time): 
    state = AeroportState()

    while state.time < time: 
        arrival_event(state)
        takeoff_event(state)

        # print()
        # Time.sleep(1)
        state.time = min([state.next_arrival_time] + [t for t in state.takeoff_time_in_path if t > state.time ])

    return state
    



In [9]:
print(aeroport_simulate(7*24*60).time_free_of_path)

[12.930714143528864, 20.02300054941046, 35.417868774924486, 54.01059186108452, 86.47831335583702]


In [21]:
def X_median(X): return sum([ x for x in X ]) / len(X)
def S_2(X):
    median = X_median(X)
    return sum([ (x - median)*(x - median) for x in X ]) / (len(X) - 1)

def stop_condition(steep, result, d):
    if steep < 30: 
        print(f'continue steep {steep}', end='\r')
        return True

    for i in range(5):
        temp_list = []
        for result_steep in result:
            temp_list.append(result_steep[i])
        s_n = sqrt(S_2(temp_list)/len(temp_list))
        if s_n > d:
            print(f'continue for stop condition {s_n} > {d}', end='\r') 
            return True
    
    print(f'finish steep {steep} with S/√n = {s_n} < {d} = d')
    return False

def run_simulations(d):
    i, result = 0, []
    while stop_condition(i, result, d):
        i += 1
        result.append(aeroport_simulate(7*24*60).time_free_of_path)
    
    return ( X_median([r[0] for r in result]),
             X_median([r[1] for r in result]),
             X_median([r[2] for r in result]),
             X_median([r[3] for r in result]),
             X_median([r[4] for r in result]))


result = run_simulations(0.2)
print(f'the path 0 is free a media of {result[0]} hours in the week')
print(f'the path 1 is free a media of {result[1]} hours in the week')
print(f'the path 2 is free a media of {result[2]} hours in the week')
print(f'the path 3 is free a media of {result[3]} hours in the week')
print(f'the path 4 is free a media of {result[4]} hours in the week')

finish steep 384 with S/√n = 0.9980247274615764 < 1 = d
the path 0 is free a media of 6.430980368069233 hours in the week
the path 1 is free a media of 16.787030952782658 hours in the week
the path 2 is free a media of 27.854032994971803 hours in the week
the path 3 is free a media of 38.98959966582595 hours in the week
the path 4 is free a media of 55.41590219585863 hours in the week
